# Generation of a polar plot

In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))


import numpy as np
import yaml
import pint
unit_registry = pint.UnitRegistry()

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from  plotly import colors

import pandas as pd

from rdkit import Chem
from rdkit.Chem import Draw, PandasTools, rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D, IPythonConsole

rdDepictor.SetPreferCoordGen(True)
from IPython.display import SVG
import rdkit

from svgutils import transform as sg

from IPython.core.display import HTML

from PLBenchmarks import targets, ligands, edges

from tqdm.notebook import tqdm
from arsenic import stats
from ipywidgets import interact

import benchmarkpl
path = benchmarkpl.__path__[0]

In [2]:
targets.set_data_dir(path)
results_dir = '10_results'

### Loading data

In [3]:
all_edges = pd.read_csv('03a_all_edges_all_ffs.csv', index_col=0)
all_edges.head()

,target,edge,ligandA,ligandB,unit,DDG_Exp.,dDDG_Exp.,DDG_OpenFF-1.0,dDDG_OpenFF-1.0,DDG_OpenFF-1.0_converged,...,error_GAFF2,abserror_GAFF2,error_cGenFF,abserror_cGenFF,error_Consensus_OpenFF_GAFF2_cGenFF,abserror_Consensus_OpenFF_GAFF2_cGenFF,error_Consensus_OpenFF_GAFF2,abserror_Consensus_OpenFF_GAFF2,error_Consensus_all,abserror_Consensus_all
jnk1_edge_17124-1_18631-1,jnk1,edge_17124-1_18631-1,17124-1,18631-1,kilocalories / mole,0.26,0.37,1.190000,0.096086,1.190000,...,1.071262,1.071262,0.516769,0.516769,0.646112,0.646112,0.784876,0.784876,1.222263,1.222263
jnk1_edge_17124-1_18634-1,jnk1,edge_17124-1_18634-1,17124-1,18634-1,kilocalories / mole,-0.33,0.29,0.580000,0.128639,0.580000,...,0.829522,0.829522,0.580956,0.580956,0.852556,0.852556,0.928604,0.928604,0.798413,0.798413
jnk1_edge_18626-1_18624-1,jnk1,edge_18626-1_18624-1,18626-1,18624-1,kilocalories / mole,0.38,0.21,0.556667,0.099301,0.556667,...,0.745717,0.745717,-0.265277,0.265277,0.309516,0.309516,0.616033,0.616033,0.446727,0.446727
jnk1_edge_18626-1_18625-1,jnk1,edge_18626-1_18625-1,18626-1,18625-1,kilocalories / mole,0.77,0.21,-0.030000,0.107462,-0.030000,...,-0.062543,0.062543,-0.294379,0.294379,-0.388337,0.388337,-0.155679,0.155679,0.143932,0.143932
jnk1_edge_18626-1_18627-1,jnk1,edge_18626-1_18627-1,18626-1,18627-1,kilocalories / mole,0.39,0.22,0.140000,0.046151,0.140000,...,0.042600,0.042600,-0.232256,0.232256,-0.020344,0.020344,0.064101,0.064101,-0.124060,0.124060


In [4]:
identifiers = [idx[4:] for idx in all_edges.columns if idx.startswith("DDG")]
identifiers

['Exp.',
 'OpenFF-1.0',
 'OpenFF-1.0_converged',
 'OpenFF-1.2',
 'OpenFF-1.2_converged',
 'OpenFF-2.0',
 'OpenFF-2.0_converged',
 'OPLS3e',
 'GAFF2',
 'cGenFF',
 'Consensus_OpenFF_GAFF2_cGenFF',
 'Consensus_OpenFF_GAFF2',
 'Consensus_all']

# Polar plot with DDG values of all targets
In the following, a summary polar plot is created

In [5]:
def add_bars(fig, theta, barwidth, limits=[-6, .5]):
    grey1 = '#E5ECF6'
    grey2 = '#8EA4C5'
    fig.add_trace(
        go.Barpolar(
            r = [ limits[1] for i in theta],
            theta = theta,
            width = barwidth,
            marker_color = 'white',
            marker_line_color = 'white',
            marker_opacity = 1,
            opacity = 1,
            showlegend=False,
                hoverinfo='skip',
        )
    )


    fig.add_trace(
        go.Barpolar(
            r = [ limits[0]-limits[1] for i in theta],
            theta = theta,
            width = 3.*barwidth,
            marker_color = grey1,
            marker_line_color = grey1,
            marker_opacity = 1,
            opacity = 1,
            showlegend=False,
                hoverinfo='skip',
        )
    ) 

    fig.add_trace(
        go.Barpolar(
            r = [ -limits[0]+limits[1] for i in theta],
            theta = theta,
            width = barwidth,
            marker_color = 'white',
            marker_line_color = 'white',
            marker_opacity = 1,
            opacity = 1,
                         showlegend=False,
                hoverinfo='skip',
        )
    )  
    
    mid_theta = (theta[:-1]+theta[1:])/2.0
    fig.add_trace(
        go.Barpolar(
            r = [ limits[0] for i in theta],
            theta = mid_theta,
            width = barwidth/2.0,
            marker_color = grey2,
            marker_line_color = grey2,
            marker_opacity = 1,
            opacity = 1,
            showlegend=False,
                hoverinfo='skip',
        )
    ) 

    fig.add_trace(
        go.Barpolar(
            r = [-limits[0]+limits[1] for i in theta],
            theta = mid_theta,
            width = barwidth/2.0,
            marker_color = grey2,
            marker_line_color = grey2,
            marker_opacity = 1,
            opacity = 1,
                         showlegend=False,
                hoverinfo='skip',
        )
    )  

In [6]:
def annotate_dev(fig, annot_angles, annot_values):
    arrow_rad = 0.51
    line_rad = 0.53
    for t, a in zip(annot_angles, annot):
        t = t/360. * 2.* np.pi
        fig.add_shape(
                type="line",
                xref="paper",
                yref="paper",
                x0=0.5,
                y0=0.5,
                x1=0.5+line_rad*np.cos(t),
                y1=0.5+line_rad*np.sin(t),
                line=dict(
                    color='black',
                    width=.5,
                ),
        )
        fig.add_annotation(
                text=a,
                xref="paper",
                yref="paper",
                x=0.5+line_rad*np.cos(t),
                y=0.5+line_rad*np.sin(t),
                xshift=0,
                yshift=0,
                xanchor='left',
                yanchor='middle',
                textangle=-t,
                showarrow=False
        )
    path = f"M {0.5+arrow_rad*np.cos(annot_angles[0]/180*np.pi*2)}, {0.5+arrow_rad*np.sin(annot_angles[0]/180*np.pi*2)}"
    path+= f" Q {.5+arrow_rad+0.01}, 0.5"
    path+= f" {0.5+arrow_rad*np.cos(annot_angles[3]/180*np.pi*2)}, {0.5+arrow_rad*np.sin(annot_angles[3]/180*np.pi*2)}"

    fig.add_annotation(
        text="",
        x = 0.5+arrow_rad*np.cos(annot_angles[3]/180*np.pi*2),
        y = 0.5+arrow_rad*np.sin(annot_angles[3]/180*np.pi*2),
                      arrowhead = 4,
                      arrowsize = 1.0,
                xref="paper",
                yref="paper",    
        ax = 10*np.sin(annot_angles[3]/180*np.pi*2),
          ay = 10*np.cos(annot_angles[3]/180*np.pi*2),
        arrowcolor = 'black',
        showarrow=True,
    )
    fig.add_annotation(
        text="\u0394\u0394\u0394G" + "<br>[kcal mol<sup>-1</sup>]",
        x = .5 + line_rad + .02,
        y = 0.5,
        xref="paper",
        yref="paper",    
                xshift=0,
                yshift=0,
                xanchor='left',
                yanchor='middle',
        showarrow=False,
    )
    fig.add_shape(
        type="path",
        path=path,
        line = dict(color='black', width=1.0),
                xref="paper",
                yref="paper",
    )
    fig.add_annotation(
        text='\u0394\u0394G<sub>exp</sub>',
        x = 0.5+.03*np.cos(annot_angles[0]/180*np.pi*2),
        y = 0.5+.03*np.sin(annot_angles[0]/180*np.pi*2),
                xref="paper",
                yref="paper",    
        showarrow=False,
        xanchor='center',
        yanchor='top',
        textangle=-annot_angles[0],
        font_size=14
    )

In [7]:
limits = [-7, .5]
segment = 360 / (len(targets.target_dict)+1)

scale = 5.5 # kcal/mol; meaning every segment has the range [-6,6]kcal/mol
threshold = 5.0
target_list = list(targets.target_dict.keys())
                
theta_list = np.array([i*segment for i in range(len(targets.target_dict)+1)])

annot_angles = np.array([-3, -1, 1, 3])/(2.0*scale)*segment
annot = ['-3', '-1', '1', '3'] 
barwidth = segment/scale

newcolors = ["#d3dc57",
"#704b89",
"#79e574",
"#820032",
"#d1db80",
"#73a0ff",
"#ffb281",
"#5f62d8",
"#005015",
"#ff6f52",
"#01ac8f",
"#b44aba",
"#488c00",
"#ac1c8c",
"#00b159",
"#ff62af",
"#345c00",
"#c98eff",
"#006e43",
"#009cda",
"#8d8200",
"#2fefb3",
"#aa0038",
"#01a76f"]   


In [10]:
def polar_plot(idx, idx2=None):
    polar_df = all_edges.copy()
    
    for i, row in polar_df.iterrows():
        if row[f'DDG_{identifiers[0]}'] > 0.0:
            polar_df.loc[i, f'error_{idx}'] *= -1.0
            if idx is not None:
                polar_df.loc[i, f'error_{idx}'] *= -1.0
            polar_df.loc[i, f'DDG_{identifiers[0]}'] *= -1.0
            polar_df.loc[i, f'ligandA'] = row['ligandB']
            polar_df.loc[i, f'ligandB'] = row['ligandA']
        
    polar_df['offset'] = polar_df['target'].apply(lambda x: target_list.index(x)) + 1.0
    polar_df['theta'] = polar_df[f'error_{idx}']
    polar_df['theta'] = polar_df['theta'].apply(lambda x: threshold+.25 if x > threshold else x)
    polar_df['theta'] = polar_df['theta'].apply(lambda x: -threshold-.25 if x < -threshold else x)
    polar_df['theta'] = polar_df['theta']/(2.0*scale)
    polar_df['theta'] *= segment
    polar_df['theta'] += polar_df['offset']*segment
    if idx2 is not None:
        polar_df['theta2'] = polar_df[f'error_{idx2}']
        polar_df['theta2'] = polar_df['theta2'].apply(lambda x: threshold+.25 if x > threshold else x)
        polar_df['theta2'] = polar_df['theta2'].apply(lambda x: -threshold-.25 if x < -threshold else x)
        polar_df['theta2'] = polar_df['theta2']/(2.0*scale)
        polar_df['theta2'] *= segment
        polar_df['theta2'] += polar_df['offset']*segment        
#     polar_df['not_converged'] = all_edges[f'DDG_{identifiers[9]}'].isna().values


        
    fig = go.Figure()
    add_bars(fig, theta_list, barwidth, limits)

    for i, target in enumerate(targets.target_dict): 
        metrics = stats.bootstrap_statistic_with_error(all_edges.loc[all_edges['target']==target,f'DDG_Exp.'].values, 
                          all_edges.loc[all_edges['target']==target,f'DDG_{idx}'].values, 
                          dy_true=all_edges.loc[all_edges['target']==target,f'dDDG_Exp.'].values, 
                          dy_pred=all_edges.loc[all_edges['target']==target,f'dDDG_{idx}'].values, 
                          statistic='MUE')
        text = [f'{row["ligandA"]}, {row["ligandB"]}<br>'\
                f'DDG_exp: {row[f"DDG_{identifiers[0]}"]:.1f}<br>'\
                f'deviation: {row[f"error_{idx}"]:.1f}' for i, row in polar_df[polar_df['target']==target].iterrows()]
        fig.add_trace(
            go.Scatterpolar(
                    r = polar_df[f'DDG_{identifiers[0]}'][polar_df['target']==target],
                    theta = polar_df['theta'][polar_df['target']==target],
                    mode = 'markers',
                    marker_opacity = .65,
                    marker_size = 5,
                    marker_line_width = 0.5,
                    text = text,
                    hovertemplate = "%{text}",
                    name=f'{target}<br>{metrics["mle"]:.1f}[{metrics["low"]:.1f},{metrics["high"]:.1f}]',
                         showlegend=False,
                )

            )
        if idx2 is not None:
            fig.add_trace(
                go.Scatterpolar(
                    r = polar_df[f'DDG_{identifiers[0]}'][polar_df['target']==target],
                    theta = polar_df['theta2'][polar_df['target']==target],
                    mode = 'markers',
                    marker_color='black',
                    marker_opacity = 1.0,
                    marker_size = 3,
                    marker_line_width = 0.0,
                    text = text,
                    hovertemplate = "%{text}",
                    name=target
                )

            )
    # target='test'
    # fig.add_trace(
    #     go.Scatterpolar(
    #             r = polar_df['r'][polar_df['target']==target],
    #             theta = polar_df['theta'][polar_df['target']==target],
    #             mode = 'markers',
    #             marker_opacity = .65,
    # #                 marker_size = 5 + .5/np.array(polar_df['yerr'][polar_df['target']==target], dtype=float),
    #             marker_line_width = 0.5,
    #             marker_color='black',
    #             name=target
    #         )
    #     )
    fig.update_layout(
        polar = dict(
            radialaxis = dict(
    #             title = r'$\Delta \Delta G_{exp}$',
                range=limits,
                tickvals = np.array(range(int(limits[0]/2)*2,2, 2)),
                angle=annot_angles[0]
            ),
            angularaxis = dict(
                tickmode = 'array',
                tickvals = [float(i*segment) for i in range(len(targets.target_dict)+1)],
                showtickprefix = 'all',
                ticktext = [""] + [target for target in targets.target_dict],
                showline = True,
                linecolor = 'white'
            ),
            hole=.2
        ),
        showlegend=False,
        colorway = newcolors,
        width=700,
        height=700,
        margin=dict(l=100, r=100, t=100, b=100)
    )

    fig.update_polars(bgcolor='#BFCEE3')
    annotate_dev(fig, annot_angles, annot)
    if idx2 is None:
        fig.write_image(f'03c_polar_openff_{idx}.png')
    else:
        fig.write_image(f'03c_polar_openff_{idx}_{idx2}.png')
        
    fig.show()

In [11]:
out = interact(polar_plot, idx=identifiers[1:], idx2=[None]+identifiers[1:])

interactive(children=(Dropdown(description='idx', options=('OpenFF-1.0', 'OpenFF-1.0_converged', 'OpenFF-1.2',…